In [6]:
# text manipulation
import re
import string

# Data management
import pandas as pd
import numpy as np
from scipy.sparse import *
import scipy

# NLP
import nltk
import nltk.collocations as collocations
from nltk.tag import tnt
import spacy
import gensim
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from collections import defaultdict
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

# sklearn
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV


# keras
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Embedding

#visualisation
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing

%matplotlib inline

In [7]:
train = pd.read_csv('./train.csv')

In [8]:
train.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [9]:
train.shape

(1306122, 3)

In [10]:
no_insincere = train[train['target']==1].target.count()
no_sincere = train[train['target']==0].target.count()

print('No. of insincere questions:', no_insincere)
print('No. of sincere questions:', no_sincere)
print('% of insincere questions:', train.target.mean())
print('Null score:', 1- train.target.mean())

No. of insincere questions: 80810
No. of sincere questions: 1225312
% of insincere questions: 0.06187017751787352
Null score: 0.9381298224821265


In [11]:
clean_questions = (re.sub("[^A-Za-z']+", ' ', q).lower() for q in train['question_text'])

In [12]:
stopwords = list(nltk.corpus.stopwords.words('english'))

In [13]:
%%time
# remove stop words and lower all characters
clean_questions = [' '.join(w for w in nltk.word_tokenize(q.lower()) if w not in stopwords) for q in clean_questions]

Wall time: 3min 39s


In [14]:
train['clean_question'] = list(clean_questions)

In [15]:
train.head()

,qid,question_text,target,clean_question
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0,quebec nationalists see province nation
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0,adopted dog would encourage people adopt shop
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0,velocity affect time velocity affect space geo...
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0,otto von guericke used magdeburg hemispheres
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0,convert montra helicon mountain bike changing ...


In [16]:
X = train['clean_question']
y = train.target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=495, test_size=0.2)

In [ ]:
INPUT_LEN = 80

tokenizer = Tokenizer(num_words = MAX_NB_WORDS)
tokenizer.fit_on_texts(X_train)

Train_sequences = tokenizer.texts_to_sequences(X_train) 
X_train_data = pad_sequences(Train_sequences, maxlen=INPUT_LEN)

Test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_data = pad_sequences(Test_sequences, maxlen=INPUT_LEN)

In [ ]:
word_index = tokenizer.word_index

In [ ]:
len(word_index)

163084

### Word2Vec embedding using training data.

In [ ]:
w2v = Word2Vec(min_count=5,
               size = 100,
               window=4,
               workers=cores-1)

In [ ]:

w2v.build_vocab(X_train_tokens, progress_per=100000)

In [ ]:
words = list(w2v.wv.vocab)
len(words)

In [ ]:
%%time
w2v.train(token_list, total_examples=w2v.corpus_count, epochs=30, report_delay=1)

In [ ]:
w2v.wv.most_similar('nice') # Testing trained embedding

In [ ]:
# Tunning parameters
MAX_NB_WORDS = 100000
EMBEDDING_DIM = 300
LSTM_UNITS = 100
DROP = 0.2
RE_DROP = 0.2

In [ ]:
%%time
embedding_matrix = np.zeros((len(word_index), EMBEDDING_DIM))
for word, i in word_index.items():
    if word in w2v.vocab:
        embedding_matrix[i] = word2vec.word_vec(word)
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))


#### LSTM

In [ ]:
## Network architecture
model = Sequential()

model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, weights = [embedding_matrix], input_length=INPUT_LEN)) 
model.add(LSTM(LSTM_UNITS, dropout=DROP, recurrent_dropout=RE_DROP))
model.add(Dense(1, activation='sigmoid')) 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train_data, y_train, validation_split=0.4, epochs=3)

In [ ]:
%%time
y_pred = model_conv.predict(X_test_data, workers=6)

In [ ]:
y_predRound = [ 1 if y >= 0.2 else 0  for y in y_pred]

In [ ]:
print('accuracy %s' % accuracy_score(y_test, y_predRound))
print(classification_report(y_test, y_predRound))
confusion_matrix(y_test, y_predRound)

#### Conv1D LSTM

In [17]:
# Tunning parameters
MAX_NB_WORDS = 100000
EMBEDDING_DIM = 200
LSTM_UNITS = 200
DROP = 0.2
RE_DROP = 0.2
FILTERS = 128
KERNEL = 1
POOL_SIZE = 4

In [ ]:
%%time
def create_conv_model():
    model_conv = Sequential()
    model_conv.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=INPUT_LEN))
    model_conv.add(Dropout(DROP))
    model_conv.add(Conv1D(FILTERS, KERNEL, activation='relu'))
    model_conv.add(MaxPooling1D(pool_size=POOL_SIZE)) # Return max value from 4 adjacent values, reduce computational requirement and reduce overfitting.
    model_conv.add(LSTM(LSTM_UNITS))
    model_conv.add(Dense(1, activation='sigmoid'))
    model_conv.compile(loss='binary_crossentropy', optimizer='adam',    metrics=['accuracy'])
    return model_conv
model_conv = create_conv_model()
model_conv.fit(X_train_data, y_train, validation_split=0.4, epochs = 3)

# Reduced computation time to 1/4.

In [ ]:
%%time
y_pred = model_conv.predict(X_test_data, workers=6)

In [ ]:
y_predRound = [ 1 if y >= 0.2 else 0  for y in y_pred]

In [ ]:
print('accuracy %s' % accuracy_score(y_test, y_predRound))
print(classification_report(y_test, y_predRound))
confusion_matrix(y_test, y_predRound)

### Using google pre-trained Word2Vec model

In [ ]:
%%time

# model pre-trained by Google
wv_google = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)
wv_google.init_sims(replace=True)

INFO - 23:26:16: loading projection weights from GoogleNews-vectors-negative300.bin.gz
INFO - 23:28:20: loaded (3000000, 300) matrix from GoogleNews-vectors-negative300.bin.gz
INFO - 23:28:20: precomputing L2-norms of word weight vectors


Wall time: 2min 35s


In [ ]:
wv_google.wv.most_similar('nice')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('good', 0.6836092472076416),
 ('lovely', 0.6676311492919922),
 ('neat', 0.6616737246513367),
 ('fantastic', 0.6569241285324097),
 ('wonderful', 0.6561347246170044),
 ('terrific', 0.6552367210388184),
 ('great', 0.6454657316207886),
 ('awesome', 0.6404187083244324),
 ('nicer', 0.6302444934844971),
 ('decent', 0.5993332266807556)]

In [ ]:
# Tunning parameters
MAX_NB_WORDS = 163084
EMBEDDING_DIM = 300
LSTM_UNITS = 100
DROP = 0.2
RE_DROP = 0.2

In [ ]:
%%time
embedding_matrix = np.zeros((len(word_index), EMBEDDING_DIM))
for word, i in word_index.items():
    if word in wv_google.vocab:
        embedding_matrix[i] = wv_google.word_vec(word)
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Null word embeddings: 91508
Wall time: 574 ms



#### LSTM

In [ ]:

#### LSTM## Network architecture
model = Sequential()

model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, weights = [embedding_matrix], input_length=INPUT_LEN)) 
model.add(LSTM(LSTM_UNITS, dropout=DROP, recurrent_dropout=RE_DROP))
model.add(Dense(1, activation='sigmoid')) 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train_data, y_train, validation_split=0.4, epochs=3)

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 626938 samples, validate on 417959 samples
Epoch 1/3


  6176/626938 [..............................] - ETA: 24:51:51 - loss: 0.6896 - acc: 0.718 - ETA: 13:07:31 - loss: 0.6837 - acc: 0.750 - ETA: 9:12:24 - loss: 0.6760 - acc: 0.802 - ETA: 7:15:54 - loss: 0.6670 - acc: 0.82 - ETA: 6:05:25 - loss: 0.6540 - acc: 0.85 - ETA: 5:21:22 - loss: 0.6452 - acc: 0.85 - ETA: 4:46:57 - loss: 0.6312 - acc: 0.87 - ETA: 4:20:47 - loss: 0.6158 - acc: 0.88 - ETA: 4:00:28 - loss: 0.6053 - acc: 0.88 - ETA: 3:44:18 - loss: 0.5853 - acc: 0.89 - ETA: 3:30:59 - loss: 0.5631 - acc: 0.89 - ETA: 3:20:25 - loss: 0.5387 - acc: 0.90 - ETA: 3:10:59 - loss: 0.5109 - acc: 0.90 - ETA: 3:02:43 - loss: 0.5168 - acc: 0.90 - ETA: 2:55:50 - loss: 0.4907 - acc: 0.90 - ETA: 2:49:50 - loss: 0.4682 - acc: 0.91 - ETA: 2:44:16 - loss: 0.4563 - acc: 0.91 - ETA: 2:39:31 - loss: 0.4391 - acc: 0.91 - ETA: 2:35:12 - loss: 0.4172 - acc: 0.91 - ETA: 2:31:19 - loss: 0.4035 - acc: 0.92 - ETA: 2:27:41 - loss: 0.3854 - acc: 0.92 - ETA: 2:24:25 - loss: 0.3689 - acc: 0.92 - ETA: 2:21:29 - loss: 0

 12384/626938 [..............................] - ETA: 1:25:48 - loss: 0.2179 - acc: 0.93 - ETA: 1:25:47 - loss: 0.2181 - acc: 0.93 - ETA: 1:25:44 - loss: 0.2177 - acc: 0.93 - ETA: 1:25:40 - loss: 0.2177 - acc: 0.93 - ETA: 1:25:37 - loss: 0.2173 - acc: 0.93 - ETA: 1:25:34 - loss: 0.2168 - acc: 0.93 - ETA: 1:25:33 - loss: 0.2163 - acc: 0.93 - ETA: 1:25:30 - loss: 0.2158 - acc: 0.93 - ETA: 1:25:28 - loss: 0.2151 - acc: 0.94 - ETA: 1:25:25 - loss: 0.2153 - acc: 0.93 - ETA: 1:25:22 - loss: 0.2151 - acc: 0.94 - ETA: 1:25:20 - loss: 0.2144 - acc: 0.94 - ETA: 1:25:18 - loss: 0.2140 - acc: 0.94 - ETA: 1:25:15 - loss: 0.2150 - acc: 0.94 - ETA: 1:25:12 - loss: 0.2154 - acc: 0.93 - ETA: 1:25:09 - loss: 0.2150 - acc: 0.93 - ETA: 1:25:07 - loss: 0.2143 - acc: 0.94 - ETA: 1:25:06 - loss: 0.2144 - acc: 0.93 - ETA: 1:25:03 - loss: 0.2137 - acc: 0.94 - ETA: 1:25:02 - loss: 0.2128 - acc: 0.94 - ETA: 1:24:59 - loss: 0.2126 - acc: 0.94 - ETA: 1:24:56 - loss: 0.2136 - acc: 0.93 - ETA: 1:24:54 - loss: 0.2137

 18592/626938 [..............................] - ETA: 1:21:38 - loss: 0.1846 - acc: 0.94 - ETA: 1:21:37 - loss: 0.1846 - acc: 0.94 - ETA: 1:21:36 - loss: 0.1845 - acc: 0.94 - ETA: 1:21:35 - loss: 0.1846 - acc: 0.94 - ETA: 1:21:34 - loss: 0.1844 - acc: 0.94 - ETA: 1:21:34 - loss: 0.1844 - acc: 0.94 - ETA: 1:21:33 - loss: 0.1840 - acc: 0.94 - ETA: 1:21:32 - loss: 0.1841 - acc: 0.94 - ETA: 1:21:31 - loss: 0.1839 - acc: 0.94 - ETA: 1:21:30 - loss: 0.1837 - acc: 0.94 - ETA: 1:21:29 - loss: 0.1836 - acc: 0.94 - ETA: 1:21:29 - loss: 0.1832 - acc: 0.94 - ETA: 1:21:28 - loss: 0.1834 - acc: 0.94 - ETA: 1:21:30 - loss: 0.1837 - acc: 0.94 - ETA: 1:21:33 - loss: 0.1835 - acc: 0.94 - ETA: 1:21:33 - loss: 0.1833 - acc: 0.94 - ETA: 1:21:31 - loss: 0.1835 - acc: 0.94 - ETA: 1:21:30 - loss: 0.1835 - acc: 0.94 - ETA: 1:21:32 - loss: 0.1835 - acc: 0.94 - ETA: 1:21:32 - loss: 0.1835 - acc: 0.94 - ETA: 1:21:31 - loss: 0.1832 - acc: 0.94 - ETA: 1:21:31 - loss: 0.1830 - acc: 0.94 - ETA: 1:21:32 - loss: 0.1828

...

219040/626938 [=========>....................] - ETA: 55:26 - loss: 0.1330 - acc: 0.94 - ETA: 55:26 - loss: 0.1330 - acc: 0.94 - ETA: 55:25 - loss: 0.1330 - acc: 0.94 - ETA: 55:25 - loss: 0.1330 - acc: 0.94 - ETA: 55:25 - loss: 0.1330 - acc: 0.94 - ETA: 55:24 - loss: 0.1330 - acc: 0.94 - ETA: 55:24 - loss: 0.1330 - acc: 0.94 - ETA: 55:24 - loss: 0.1329 - acc: 0.94 - ETA: 55:24 - loss: 0.1329 - acc: 0.94 - ETA: 55:23 - loss: 0.1329 - acc: 0.94 - ETA: 55:23 - loss: 0.1329 - acc: 0.94 - ETA: 55:23 - loss: 0.1329 - acc: 0.94 - ETA: 55:23 - loss: 0.1329 - acc: 0.94 - ETA: 55:22 - loss: 0.1329 - acc: 0.94 - ETA: 55:22 - loss: 0.1329 - acc: 0.94 - ETA: 55:22 - loss: 0.1329 - acc: 0.94 - ETA: 55:22 - loss: 0.1329 - acc: 0.94 - ETA: 55:21 - loss: 0.1329 - acc: 0.94 - ETA: 55:21 - loss: 0.1329 - acc: 0.94 - ETA: 55:21 - loss: 0.1329 - acc: 0.94 - ETA: 55:20 - loss: 0.1329 - acc: 0.94 - ETA: 55:20 - loss: 0.1329 - acc: 0.94 - ETA: 55:20 - loss: 0.1329 - acc: 0.94 - ETA: 55:20 - loss: 0.1329 - acc

225568/626938 [=========>....................] - ETA: 54:28 - loss: 0.1325 - acc: 0.94 - ETA: 54:28 - loss: 0.1325 - acc: 0.94 - ETA: 54:27 - loss: 0.1325 - acc: 0.94 - ETA: 54:27 - loss: 0.1325 - acc: 0.94 - ETA: 54:27 - loss: 0.1325 - acc: 0.94 - ETA: 54:26 - loss: 0.1325 - acc: 0.94 - ETA: 54:26 - loss: 0.1325 - acc: 0.94 - ETA: 54:26 - loss: 0.1325 - acc: 0.94 - ETA: 54:26 - loss: 0.1325 - acc: 0.94 - ETA: 54:26 - loss: 0.1325 - acc: 0.94 - ETA: 54:25 - loss: 0.1325 - acc: 0.94 - ETA: 54:25 - loss: 0.1325 - acc: 0.94 - ETA: 54:25 - loss: 0.1325 - acc: 0.94 - ETA: 54:25 - loss: 0.1325 - acc: 0.94 - ETA: 54:25 - loss: 0.1325 - acc: 0.94 - ETA: 54:24 - loss: 0.1325 - acc: 0.94 - ETA: 54:24 - loss: 0.1325 - acc: 0.94 - ETA: 54:24 - loss: 0.1325 - acc: 0.94 - ETA: 54:23 - loss: 0.1324 - acc: 0.94 - ETA: 54:23 - loss: 0.1324 - acc: 0.94 - ETA: 54:23 - loss: 0.1324 - acc: 0.94 - ETA: 54:23 - loss: 0.1324 - acc: 0.94 - ETA: 54:22 - loss: 0.1324 - acc: 0.94 - ETA: 54:22 - loss: 0.1324 - acc

232096/626938 [==========>...................] - ETA: 53:37 - loss: 0.1322 - acc: 0.94 - ETA: 53:37 - loss: 0.1321 - acc: 0.94 - ETA: 53:36 - loss: 0.1321 - acc: 0.94 - ETA: 53:36 - loss: 0.1321 - acc: 0.94 - ETA: 53:36 - loss: 0.1321 - acc: 0.94 - ETA: 53:36 - loss: 0.1321 - acc: 0.94 - ETA: 53:35 - loss: 0.1321 - acc: 0.94 - ETA: 53:35 - loss: 0.1321 - acc: 0.94 - ETA: 53:35 - loss: 0.1321 - acc: 0.94 - ETA: 53:35 - loss: 0.1321 - acc: 0.94 - ETA: 53:34 - loss: 0.1321 - acc: 0.94 - ETA: 53:34 - loss: 0.1321 - acc: 0.94 - ETA: 53:34 - loss: 0.1321 - acc: 0.94 - ETA: 53:34 - loss: 0.1321 - acc: 0.94 - ETA: 53:33 - loss: 0.1321 - acc: 0.94 - ETA: 53:33 - loss: 0.1321 - acc: 0.94 - ETA: 53:33 - loss: 0.1321 - acc: 0.94 - ETA: 53:33 - loss: 0.1321 - acc: 0.94 - ETA: 53:32 - loss: 0.1321 - acc: 0.94 - ETA: 53:32 - loss: 0.1321 - acc: 0.94 - ETA: 53:32 - loss: 0.1321 - acc: 0.94 - ETA: 53:32 - loss: 0.1321 - acc: 0.94 - ETA: 53:32 - loss: 0.1321 - acc: 0.94 - ETA: 53:31 - loss: 0.1321 - acc

238624/626938 [==========>...................] - ETA: 52:47 - loss: 0.1321 - acc: 0.94 - ETA: 52:47 - loss: 0.1321 - acc: 0.94 - ETA: 52:46 - loss: 0.1321 - acc: 0.94 - ETA: 52:46 - loss: 0.1321 - acc: 0.94 - ETA: 52:46 - loss: 0.1321 - acc: 0.94 - ETA: 52:45 - loss: 0.1321 - acc: 0.94 - ETA: 52:45 - loss: 0.1321 - acc: 0.94 - ETA: 52:45 - loss: 0.1321 - acc: 0.94 - ETA: 52:45 - loss: 0.1321 - acc: 0.94 - ETA: 52:44 - loss: 0.1321 - acc: 0.94 - ETA: 52:44 - loss: 0.1320 - acc: 0.94 - ETA: 52:44 - loss: 0.1320 - acc: 0.94 - ETA: 52:44 - loss: 0.1320 - acc: 0.94 - ETA: 52:43 - loss: 0.1320 - acc: 0.94 - ETA: 52:43 - loss: 0.1320 - acc: 0.94 - ETA: 52:43 - loss: 0.1321 - acc: 0.94 - ETA: 52:42 - loss: 0.1321 - acc: 0.94 - ETA: 52:42 - loss: 0.1321 - acc: 0.94 - ETA: 52:42 - loss: 0.1321 - acc: 0.94 - ETA: 52:42 - loss: 0.1321 - acc: 0.94 - ETA: 52:41 - loss: 0.1321 - acc: 0.94 - ETA: 52:41 - loss: 0.1320 - acc: 0.94 - ETA: 52:41 - loss: 0.1321 - acc: 0.94 - ETA: 52:40 - loss: 0.1321 - acc

240576/626938 [==========>...................] - ETA: 51:49 - loss: 0.1320 - acc: 0.94 - ETA: 51:49 - loss: 0.1320 - acc: 0.94 - ETA: 51:48 - loss: 0.1320 - acc: 0.94 - ETA: 51:48 - loss: 0.1320 - acc: 0.94 - ETA: 51:48 - loss: 0.1320 - acc: 0.94 - ETA: 51:47 - loss: 0.1320 - acc: 0.94 - ETA: 51:47 - loss: 0.1320 - acc: 0.94 - ETA: 51:47 - loss: 0.1320 - acc: 0.94 - ETA: 51:47 - loss: 0.1320 - acc: 0.94 - ETA: 51:46 - loss: 0.1320 - acc: 0.94 - ETA: 51:46 - loss: 0.1320 - acc: 0.94 - ETA: 51:46 - loss: 0.1320 - acc: 0.94 - ETA: 51:45 - loss: 0.1320 - acc: 0.94 - ETA: 51:45 - loss: 0.1320 - acc: 0.94 - ETA: 51:45 - loss: 0.1320 - acc: 0.94 - ETA: 51:44 - loss: 0.1320 - acc: 0.94 - ETA: 51:44 - loss: 0.1319 - acc: 0.94 - ETA: 51:44 - loss: 0.1319 - acc: 0.94 - ETA: 51:44 - loss: 0.1320 - acc: 0.94 - ETA: 51:43 - loss: 0.1319 - acc: 0.94 - ETA: 51:43 - loss: 0.1319 - acc: 0.94 - ETA: 51:43 - loss: 0.1319 - acc: 0.94 - ETA: 51:42 - loss: 0.1319 - acc: 0.94 - ETA: 51:42 - loss: 0.1319 - acc

In [ ]:
%%time
y_pred = model_conv.predict(X_test_data, workers=6)

In [ ]:
y_predRound = [ 1 if y >= 0.2 else 0  for y in y_pred]

In [ ]:
print('accuracy %s' % accuracy_score(y_test, y_predRound))
print(classification_report(y_test, y_predRound))
confusion_matrix(y_test, y_predRound)

#### Conv1D LSTM

In [41]:
# Tunning parameters
MAX_NB_WORDS = 100000
EMBEDDING_DIM = 200
LSTM_UNITS = 200
DROP = 0.2
RE_DROP = 0.2
FILTERS = 128
KERNEL = 1
POOL_SIZE = 4

In [46]:
%%time
def create_conv_model():
    model_conv = Sequential()
    model_conv.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=INPUT_LEN))
    model_conv.add(Dropout(DROP))
    model_conv.add(Conv1D(FILTERS, KERNEL, activation='relu'))
    model_conv.add(MaxPooling1D(pool_size=POOL_SIZE)) # Return max value from 4 adjacent values, reduce computational requirement and reduce overfitting.
    model_conv.add(LSTM(LSTM_UNITS))
    model_conv.add(Dense(1, activation='sigmoid'))
    model_conv.compile(loss='binary_crossentropy', optimizer='adam',    metrics=['accuracy'])
    return model_conv

model_conv = create_conv_model()
model_conv.fit(X_train_data, y_train, validation_split=0.4, epochs = 3)

# Reduced computation time to 1/4.

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 626938 samples, validate on 417959 samples
Epoch 1/3


  6464/626938 [..............................] - ETA: 8:33:36 - loss: 0.6992 - acc: 0.06 - ETA: 4:27:23 - loss: 0.6805 - acc: 0.46 - ETA: 3:04:34 - loss: 0.6586 - acc: 0.61 - ETA: 2:23:14 - loss: 0.6315 - acc: 0.69 - ETA: 1:58:22 - loss: 0.5935 - acc: 0.75 - ETA: 1:41:44 - loss: 0.5692 - acc: 0.77 - ETA: 1:29:51 - loss: 0.5365 - acc: 0.79 - ETA: 1:20:59 - loss: 0.4986 - acc: 0.81 - ETA: 1:14:16 - loss: 0.4886 - acc: 0.82 - ETA: 1:08:47 - loss: 0.4948 - acc: 0.82 - ETA: 1:04:13 - loss: 0.4624 - acc: 0.84 - ETA: 1:00:27 - loss: 0.4355 - acc: 0.85 - ETA: 57:15 - loss: 0.4132 - acc: 0.8606 - ETA: 54:29 - loss: 0.3884 - acc: 0.87 - ETA: 52:04 - loss: 0.3670 - acc: 0.87 - ETA: 49:59 - loss: 0.3532 - acc: 0.88 - ETA: 48:11 - loss: 0.3513 - acc: 0.88 - ETA: 46:37 - loss: 0.3393 - acc: 0.89 - ETA: 45:08 - loss: 0.3391 - acc: 0.89 - ETA: 43:49 - loss: 0.3243 - acc: 0.89 - ETA: 42:40 - loss: 0.3202 - acc: 0.89 - ETA: 41:38 - loss: 0.3250 - acc: 0.89 - ETA: 40:39 - loss: 0.3129 - acc: 0.90 - ETA: 

 19520/626938 [..............................] - ETA: 19:58 - loss: 0.2116 - acc: 0.93 - ETA: 19:58 - loss: 0.2113 - acc: 0.93 - ETA: 19:58 - loss: 0.2109 - acc: 0.93 - ETA: 19:58 - loss: 0.2108 - acc: 0.93 - ETA: 19:57 - loss: 0.2107 - acc: 0.93 - ETA: 19:57 - loss: 0.2105 - acc: 0.93 - ETA: 19:57 - loss: 0.2104 - acc: 0.93 - ETA: 19:57 - loss: 0.2103 - acc: 0.93 - ETA: 19:57 - loss: 0.2101 - acc: 0.93 - ETA: 19:56 - loss: 0.2098 - acc: 0.93 - ETA: 19:56 - loss: 0.2098 - acc: 0.93 - ETA: 19:56 - loss: 0.2095 - acc: 0.93 - ETA: 19:56 - loss: 0.2097 - acc: 0.93 - ETA: 19:56 - loss: 0.2095 - acc: 0.93 - ETA: 19:56 - loss: 0.2096 - acc: 0.93 - ETA: 19:56 - loss: 0.2092 - acc: 0.93 - ETA: 19:56 - loss: 0.2089 - acc: 0.93 - ETA: 19:56 - loss: 0.2088 - acc: 0.93 - ETA: 19:55 - loss: 0.2084 - acc: 0.93 - ETA: 19:55 - loss: 0.2082 - acc: 0.93 - ETA: 19:55 - loss: 0.2082 - acc: 0.93 - ETA: 19:55 - loss: 0.2079 - acc: 0.93 - ETA: 19:54 - loss: 0.2077 - acc: 0.93 - ETA: 19:54 - loss: 0.2080 - acc

 26048/626938 [>.............................] - ETA: 19:22 - loss: 0.1919 - acc: 0.93 - ETA: 19:22 - loss: 0.1916 - acc: 0.93 - ETA: 19:22 - loss: 0.1915 - acc: 0.93 - ETA: 19:22 - loss: 0.1913 - acc: 0.93 - ETA: 19:22 - loss: 0.1911 - acc: 0.93 - ETA: 19:22 - loss: 0.1911 - acc: 0.93 - ETA: 19:22 - loss: 0.1909 - acc: 0.93 - ETA: 19:21 - loss: 0.1906 - acc: 0.93 - ETA: 19:21 - loss: 0.1908 - acc: 0.93 - ETA: 19:21 - loss: 0.1907 - acc: 0.93 - ETA: 19:21 - loss: 0.1907 - acc: 0.93 - ETA: 19:21 - loss: 0.1909 - acc: 0.93 - ETA: 19:21 - loss: 0.1911 - acc: 0.93 - ETA: 19:20 - loss: 0.1909 - acc: 0.93 - ETA: 19:20 - loss: 0.1911 - acc: 0.93 - ETA: 19:20 - loss: 0.1909 - acc: 0.93 - ETA: 19:20 - loss: 0.1910 - acc: 0.93 - ETA: 19:20 - loss: 0.1908 - acc: 0.93 - ETA: 19:20 - loss: 0.1905 - acc: 0.93 - ETA: 19:20 - loss: 0.1906 - acc: 0.93 - ETA: 19:19 - loss: 0.1904 - acc: 0.93 - ETA: 19:19 - loss: 0.1906 - acc: 0.93 - ETA: 19:19 - loss: 0.1907 - acc: 0.93 - ETA: 19:19 - loss: 0.1905 - acc

...

600896/626938 [===========================>..] - ETA: 1:00 - loss: 0.1009 - acc: 0.960 - ETA: 1:00 - loss: 0.1009 - acc: 0.960 - ETA: 1:00 - loss: 0.1009 - acc: 0.960 - ETA: 1:00 - loss: 0.1009 - acc: 0.960 - ETA: 1:00 - loss: 0.1009 - acc: 0.960 - ETA: 1:00 - loss: 0.1009 - acc: 0.960 - ETA: 1:00 - loss: 0.1009 - acc: 0.960 - ETA: 1:00 - loss: 0.1009 - acc: 0.960 - ETA: 59s - loss: 0.1009 - acc: 0.960 - ETA: 59s - loss: 0.1009 - acc: 0.96 - ETA: 59s - loss: 0.1009 - acc: 0.96 - ETA: 59s - loss: 0.1009 - acc: 0.96 - ETA: 59s - loss: 0.1009 - acc: 0.96 - ETA: 59s - loss: 0.1009 - acc: 0.96 - ETA: 59s - loss: 0.1009 - acc: 0.96 - ETA: 59s - loss: 0.1009 - acc: 0.96 - ETA: 59s - loss: 0.1009 - acc: 0.96 - ETA: 59s - loss: 0.1009 - acc: 0.96 - ETA: 59s - loss: 0.1009 - acc: 0.96 - ETA: 59s - loss: 0.1009 - acc: 0.96 - ETA: 59s - loss: 0.1009 - acc: 0.96 - ETA: 59s - loss: 0.1009 - acc: 0.96 - ETA: 59s - loss: 0.1009 - acc: 0.96 - ETA: 59s - loss: 0.1009 - acc: 0.96 - ETA: 59s - loss: 0.100

607776/626938 [============================>.] - ETA: 47s - loss: 0.1010 - acc: 0.96 - ETA: 47s - loss: 0.1010 - acc: 0.96 - ETA: 47s - loss: 0.1010 - acc: 0.96 - ETA: 47s - loss: 0.1010 - acc: 0.96 - ETA: 47s - loss: 0.1010 - acc: 0.96 - ETA: 47s - loss: 0.1010 - acc: 0.96 - ETA: 47s - loss: 0.1010 - acc: 0.96 - ETA: 47s - loss: 0.1010 - acc: 0.96 - ETA: 47s - loss: 0.1010 - acc: 0.96 - ETA: 47s - loss: 0.1010 - acc: 0.96 - ETA: 47s - loss: 0.1010 - acc: 0.96 - ETA: 47s - loss: 0.1010 - acc: 0.96 - ETA: 47s - loss: 0.1010 - acc: 0.96 - ETA: 47s - loss: 0.1010 - acc: 0.96 - ETA: 47s - loss: 0.1010 - acc: 0.96 - ETA: 46s - loss: 0.1010 - acc: 0.96 - ETA: 46s - loss: 0.1010 - acc: 0.96 - ETA: 46s - loss: 0.1010 - acc: 0.96 - ETA: 46s - loss: 0.1010 - acc: 0.96 - ETA: 46s - loss: 0.1010 - acc: 0.96 - ETA: 46s - loss: 0.1010 - acc: 0.96 - ETA: 46s - loss: 0.1010 - acc: 0.96 - ETA: 46s - loss: 0.1010 - acc: 0.96 - ETA: 46s - loss: 0.1010 - acc: 0.96 - ETA: 46s - loss: 0.1010 - acc: 0.96 - E

614656/626938 [============================>.] - ETA: 35s - loss: 0.1011 - acc: 0.96 - ETA: 35s - loss: 0.1011 - acc: 0.96 - ETA: 35s - loss: 0.1011 - acc: 0.96 - ETA: 35s - loss: 0.1011 - acc: 0.96 - ETA: 34s - loss: 0.1011 - acc: 0.96 - ETA: 34s - loss: 0.1011 - acc: 0.96 - ETA: 34s - loss: 0.1011 - acc: 0.96 - ETA: 34s - loss: 0.1011 - acc: 0.96 - ETA: 34s - loss: 0.1011 - acc: 0.96 - ETA: 34s - loss: 0.1011 - acc: 0.96 - ETA: 34s - loss: 0.1011 - acc: 0.96 - ETA: 34s - loss: 0.1011 - acc: 0.96 - ETA: 34s - loss: 0.1011 - acc: 0.96 - ETA: 34s - loss: 0.1011 - acc: 0.96 - ETA: 34s - loss: 0.1011 - acc: 0.96 - ETA: 34s - loss: 0.1011 - acc: 0.96 - ETA: 34s - loss: 0.1011 - acc: 0.96 - ETA: 34s - loss: 0.1011 - acc: 0.96 - ETA: 34s - loss: 0.1011 - acc: 0.96 - ETA: 34s - loss: 0.1011 - acc: 0.96 - ETA: 34s - loss: 0.1011 - acc: 0.96 - ETA: 33s - loss: 0.1011 - acc: 0.96 - ETA: 33s - loss: 0.1011 - acc: 0.96 - ETA: 33s - loss: 0.1011 - acc: 0.96 - ETA: 33s - loss: 0.1011 - acc: 0.96 - E

621536/626938 [============================>.] - ETA: 22s - loss: 0.1012 - acc: 0.96 - ETA: 22s - loss: 0.1012 - acc: 0.96 - ETA: 22s - loss: 0.1012 - acc: 0.96 - ETA: 22s - loss: 0.1012 - acc: 0.96 - ETA: 22s - loss: 0.1012 - acc: 0.96 - ETA: 22s - loss: 0.1012 - acc: 0.96 - ETA: 22s - loss: 0.1012 - acc: 0.96 - ETA: 22s - loss: 0.1012 - acc: 0.96 - ETA: 22s - loss: 0.1012 - acc: 0.96 - ETA: 22s - loss: 0.1012 - acc: 0.96 - ETA: 21s - loss: 0.1012 - acc: 0.96 - ETA: 21s - loss: 0.1012 - acc: 0.96 - ETA: 21s - loss: 0.1012 - acc: 0.96 - ETA: 21s - loss: 0.1012 - acc: 0.96 - ETA: 21s - loss: 0.1012 - acc: 0.96 - ETA: 21s - loss: 0.1012 - acc: 0.96 - ETA: 21s - loss: 0.1012 - acc: 0.96 - ETA: 21s - loss: 0.1012 - acc: 0.96 - ETA: 21s - loss: 0.1012 - acc: 0.96 - ETA: 21s - loss: 0.1012 - acc: 0.96 - ETA: 21s - loss: 0.1012 - acc: 0.96 - ETA: 21s - loss: 0.1012 - acc: 0.96 - ETA: 21s - loss: 0.1012 - acc: 0.96 - ETA: 21s - loss: 0.1012 - acc: 0.96 - ETA: 21s - loss: 0.1012 - acc: 0.96 - E

626938/626938 [==============================] - ETA: 9s - loss: 0.1012 - acc: 0.960 - ETA: 9s - loss: 0.1012 - acc: 0.960 - ETA: 9s - loss: 0.1012 - acc: 0.960 - ETA: 9s - loss: 0.1012 - acc: 0.960 - ETA: 9s - loss: 0.1012 - acc: 0.960 - ETA: 9s - loss: 0.1012 - acc: 0.960 - ETA: 9s - loss: 0.1012 - acc: 0.960 - ETA: 9s - loss: 0.1012 - acc: 0.960 - ETA: 9s - loss: 0.1012 - acc: 0.960 - ETA: 9s - loss: 0.1012 - acc: 0.960 - ETA: 9s - loss: 0.1012 - acc: 0.960 - ETA: 9s - loss: 0.1012 - acc: 0.960 - ETA: 9s - loss: 0.1012 - acc: 0.960 - ETA: 9s - loss: 0.1012 - acc: 0.960 - ETA: 9s - loss: 0.1012 - acc: 0.960 - ETA: 8s - loss: 0.1012 - acc: 0.960 - ETA: 8s - loss: 0.1012 - acc: 0.960 - ETA: 8s - loss: 0.1012 - acc: 0.960 - ETA: 8s - loss: 0.1012 - acc: 0.960 - ETA: 8s - loss: 0.1012 - acc: 0.960 - ETA: 8s - loss: 0.1012 - acc: 0.960 - ETA: 8s - loss: 0.1012 - acc: 0.960 - ETA: 8s - loss: 0.1012 - acc: 0.960 - ETA: 8s - loss: 0.1012 - acc: 0.960 - ETA: 8s - loss: 0.1012 - acc: 0.960 - E

In [47]:
%%time
y_pred = model_conv.predict(X_test_data, workers=6)

Wall time: 2min 15s


In [60]:
y_predRound = [ 1 if y >= 0.1 else 0  for y in y_pred]

In [61]:
print('accuracy %s' % accuracy_score(y_test, y_predRound))
print(classification_report(y_test, y_predRound))
confusion_matrix(y_test, y_predRound)

accuracy 0.9267910804861709
              precision    recall  f1-score   support

           0       0.99      0.94      0.96    245063
           1       0.45      0.79      0.57     16162

   micro avg       0.93      0.93      0.93    261225
   macro avg       0.72      0.86      0.77    261225
weighted avg       0.95      0.93      0.94    261225



array([[229313,  15750],
       [  3374,  12788]], dtype=int64)